In [1]:
import numpy as np
import cv2
import keras
from yolo3_one_file_to_detect_them_all import WeightReader,make_yolov3_model,decode_netout,correct_yolo_boxes,do_nms
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy,Precision,Recall
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
from threading import Thread, Condition
from sklearn.metrics import classification_report
import time
import random
import pandas as pd

In [2]:
# check the video frames
video = cv2.VideoCapture('clip.mp4')
print(str(video.get(cv2.CAP_PROP_FPS)))
print(str(video.get(cv2.CAP_PROP_FRAME_COUNT)))
print(str(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(str(video.get(cv2.CAP_PROP_FRAME_WIDTH)))

30.0
900.0
576.0
428.0


In [ ]:
# load the weights and save to a model
model = make_yolov3_model()                     # get a yolo model
weight_reader = WeightReader('yolov3.weights')  # get the weights
weight_reader.load_weights(model)               # load the weights
model.save('Tiny_yolo.h5')                      # save the model

In [3]:
dataFrame = {'Frame':[],
        'Sedan':[],
        'SUV':[],
        'Total':[]}

In [6]:
# create train,test and validation sets
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
validate_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator()

train_generator=train_datagen.flow_from_directory('images/train',
                                                 target_size=(416,416),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
validation_generator = validate_datagen.flow_from_directory(
        'images/validation',
        target_size=(416, 416),
        batch_size=32,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory('images/test',
                                                 target_size=(416,416),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 654 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 225 images belonging to 2 classes.


In [ ]:
pretrained_model = MobileNet(weights='imagenet',include_top=False) # load MobileNet with weights from ImageNet without the last layer
pretrained_model.trainable = False                                 # make the base model untrainable
x = pretrained_model.output                                        # take the output layer
x = GlobalAveragePooling2D()(x)                                    # add a pooling layer
x = Dense(1024,activation='relu')(x)                               # add a dense layer
x = Dense(1024,activation='relu')(x)                               # add a dense layer
x = Dense(512,activation='relu')(x)                                # add a dense layer
pred_layer = Dense(2,activation='softmax')(x)                      # add a softmax layer
classification_model = Model(inputs=pretrained_model.input,outputs=pred_layer)  # initialise the model
classification_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',Precision(),
                       Recall()])                               # compile the model
# Adam optimizer 
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
for layer in classification_model.layers[:20]:
    layer.trainable=False                                       # set some layers as not trainable
for layer in classification_model.layers[20:]:
    layer.trainable=True                                        # set the rest of the layers as trainable           

step_size_train=train_generator.n//train_generator.batch_size
history = classification_model.fit(
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=10,
        validation_data=validation_generator)
classification_model.save('classification_model_latest.h5')

In [8]:
# load and prepare an image

class ObjectDetector():
    
    def __init__(self,model=[]):
        super(ObjectDetector, self).__init__()
        self.model = load_model('Tiny_yolo.h5')                                   # load the tinyYolo model
        self.classification_model = load_model('classification_model_latest.h5')
        self.input_w, self.input_h = 416, 416
        self.video = cv2.VideoCapture('clip.mp4')                      # get the input video
        self.video_w = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.video_h = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        format_video = cv2.VideoWriter_fourcc(*'MP4V')
        self.output_video = cv2.VideoWriter('output.mp4', format_video, 10, (self.video_w,self.video_h))
    
    def load_image_pixels(self, frame): 
        image = cv2.resize(frame,(self.input_w,self.input_h))
        # make prediction
        image = img_to_array(image)
        # scale pixel values to [0, 1]
        image = image.astype('float32')
        image /= 255.0
        # add a dimension so that we have one sample
        image = expand_dims(image, 0)
        return image

    def get_boxes(self,boxes, labels, thresh):
        v_boxes, v_labels, v_scores = list(), list(), list()
        # enumerate all boxes
        for box in boxes:
            # enumerate all possible labels
            for i in range(len(labels)):
                # check if the threshold for this label is high enough
                if box.classes[i] > thresh:
                    v_boxes.append(box)
                    v_labels.append(labels[i])
                    v_scores.append(box.classes[i]*100)
                    # don't break, many labels may trigger for one box            
        return v_boxes, v_labels, v_scores

    def draw_boxes(self,image, box, labels, obj_thresh):
        for i in range(len(labels)):
            label = labels[i]
            #print(labels[i])

        if len(label) > 0:
            cv2.rectangle(image, (box.xmin,box.ymin), (box.xmax,box.ymax), (0,255,0), 3) # draw a rectangle
            cv2.putText(image,                                                           # write text
                        label, 
                        (box.xmin, box.ymin - 13), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1e-3 * image.shape[0], 
                        (0,255,0), 2)

        return image
    
    def classify(self,frame,box):
        #print('inside classify')
        #time.sleep(random.random())
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        crop_image = frame[y1:y1+height,x1:x1+width]
        try:
           crop_image = cv2.resize(crop_image,(416,416))
        except:
           #print('return') 
           return frame
        crop_image = img_to_array(crop_image)
        # scale pixel values to [0, 1]
        crop_image = crop_image.astype('float32')
        crop_image /= 255.0
        # add a dimension so that we have one sample
        #pyplot.imshow(crop_image)
        crop_image = expand_dims(crop_image, 0)
        y_ = np.argmax(self.classification_model.predict(crop_image))
        if(train_generator.class_indices['sedan'] == y_):
            label = ['Sedan']
        else:
            label = ['SUV']
        image = self.draw_boxes(frame,box,label,0.6)
        return image
            
    
    def findCars(self):
        global queue
        dataFrame['Frame'] = []
        for i in range(900):
            print('Frame number is',i)    # for debug
            frameCount = dataFrame['Frame']
            dataFrame['Total'].append(0)
            if len(frameCount) > 0:
                count = frameCount[-1]
                frameCount.append(count+1)
                #print(frameCount)
            else:
                frameCount.insert(0,1)
                #print(frameCount)
            retVal,frame = self.video.read()
            # define the anchors
            anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
            # define the probability threshold for detected objects
            class_threshold = 0.6
            boxes = list()
            image_h, image_w, _ = frame.shape 
            image = self.load_image_pixels(frame)
            yhat = self.model.predict(image)

            for j in range(len(yhat)):
               # decode the output of the network
               boxes += decode_netout(yhat[j][0], anchors[j], class_threshold, self.input_h, self.input_w)
               # correct the sizes of the bounding boxes for the shape of the image
            correct_yolo_boxes(boxes, image_h, image_w, self.input_h, self.input_w)
            # suppress non-maximal boxes
            do_nms(boxes, 0.5)
            # define the labels
            labels = ["person", "bicycle", "car"]
            # get the details of the detected objects
            v_boxes, v_labels, v_scores = self.get_boxes(boxes, labels, class_threshold)
            # summarize what we found
            #print(v_boxes, v_labels, v_scores)
            #print('boxes count',len(v_boxes))
            total = 0
            #print('boxes is',v_boxes)
            for k in range(len(v_boxes)):
                box = v_boxes[k]
                #print('inside k')
                total += 1
                dataFrame['Total'][i] = total
                frame = self.classify(frame,box)
            self.output_video.write(frame)
        self.output_video.release()
        cv2.destroyAllWindows()
                        
ObjectDetector().findCars()
#o.start()


Frame number is 0
Frame number is 1
Frame number is 2
Frame number is 3


KeyboardInterrupt: 

In [ ]:
help(history)

In [ ]:
classification_model.save('classification_model_latest.h5')

In [ ]:
train_generator.class_indices

In [ ]:
import DuckDuckGoImages as ddg
ddg.download('suv',folder='images/SUV',max_urls=100)    # api for downloading images 

In [ ]:
help(cv2.resize)

In [ ]:
help(classification_model.outputs.index)

In [ ]:
millis = int(round(time.time()))
print(millis)